In [1]:
from syft import PhiTensor, GammaTensor, DataSubjectList, lazyrepeatarray

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

child = (50, 50)
DSL = ((50, 50))
GT DSL = ((3, 50, 50)) -> (100, 50, 50)


sum()
child.sum()=1
dsl.sum()= 7500,1 -> .unique() 3, 1

mul()
(n, 50, 50) 
(m, 50, 50) 
-> DSL.combine_dsl(): (n+m - overlap(n,m), 50, 50)


[50 x 50]
[50 x 50] 1-> integer, 2499-> Nans
..
..
..

gt2 = gamma_tensor().sum()
pt[0,0] = gt2

n = max(unique_ds for any 1 data point)


data subjects of (0, 0) in result =  (data subjects of row 0 in A ) + (data subjects of column 0 in B)
data subjects of (0, 1) in result =  (data subjects of row 0 in A ) + (data subjects of column 1 in B)
data subjects of (0, 2) in result =  (data subjects of row 0 in A ) + (data subjects of column 2 in B)

sum(target_shape) -> reshape






dsl[:, 0] -> (3, 50)

In [3]:
def dot(dsl1: DataSubjectList, dsl2: DataSubjectList):
    """
    a/b: 
    0D/0D -> multiplication
    1D/1D -> multiplication
    1D/other -> sum product over last axis 
    2D/2D -> matmul
    higher -> sum product over axis=-1 of a, axis=-2 of b
    
    dot(a, b)[i,j,k,m] = sum(a[i,j,:] * b[k,:,m])
    """
    
    dsl1_target_shape = (*dsl1.shape[1:-1], 1, 1)
    dsl2_target_shape = (1, 1, *dsl2.shape[1:-2], dsl2.shape[-1])
    
    print("Target shapes: ", dsl1_target_shape, dsl2_target_shape)

    summed_dsl1 = dsl1.sum(target_shape=dsl1_target_shape)
    summed_dsl2 = dsl2.sum(target_shape=dsl2_target_shape)
    print("summed shapes: ", summed_dsl1.shape, summed_dsl2.shape)

    # We need to project these data subject arrays to their entire row/column respectively
    
    dsl1_projection = np.ones((*summed_dsl1.shape[:-2], *summed_dsl2.shape[-2:])) 
    dsl2_projection = np.ones((summed_dsl2.shape[0], *summed_dsl1.shape[1:-2], *summed_dsl2.shape[-2:]))  #*summed_dsl2.shape[1:-2],
    projection = np.ones((*summed_dsl1.shape[:-2], *summed_dsl2.shape[-2:]))
    print(projection.shape)
    print("Projection shape: ", dsl1_projection.shape, dsl2_projection.shape)
    
    summed_dsl1.data_subjects_indexed = dsl1_projection * summed_dsl1.data_subjects_indexed
    summed_dsl2.data_subjects_indexed = dsl2_projection * summed_dsl2.data_subjects_indexed
    
    print("summed shapes after projection: ", summed_dsl1.shape, summed_dsl2.shape)
    
    output_ds = DataSubjectList.combine_dsi(summed_dsl1, summed_dsl2)
    print("output shape: ", output_ds.shape)
    
    # This gets rid of redundant (repeating) DSL slices.
    output_ds.data_subjects_indexed = np.unique(output_ds.data_subjects_indexed, axis=0)
    print("output shape after np.unique: ", output_ds.shape)
    return output_ds

In [4]:
np.ones((46, 54)).sum(axis=0).shape

(54,)

In [5]:
np.dot(np.ones((3,4,6,54)), np.ones((3,10,54,7))).shape

(3, 4, 6, 3, 10, 7)

In [6]:
from syft import GammaTensor, lazyrepeatarray

def test_dot():
    
    # (2, 3, 4, 54)
    gt1 = GammaTensor(
        child=np.random.random((3, 4, 54)), 
        data_subjects=DataSubjectList.from_objs(
            np.concatenate((
                np.random.choice(["Ishan"], (1, 3, 4, 54)),
                np.random.choice(["John Cena"], (1, 3, 4, 54))
            )),
        ),
        min_vals=lazyrepeatarray(0, (3, 4, 54)) , 
        max_vals=lazyrepeatarray(1, (3, 4, 54))
    )
    
    # (2, 3, 54, 7)
    gt2 = GammaTensor(
        child=np.random.random((3, 54, 7)), 
        data_subjects=DataSubjectList.from_objs(
            np.concatenate(( 
                np.random.choice(["Hulk Hogan"], (1, 3, 54, 7)),
                np.random.choice(["Potato"], (1, 3, 54, 7))
            )),
        ),
        min_vals=lazyrepeatarray(0, (3, 54, 7)) , 
        max_vals=lazyrepeatarray(1, (3, 54, 7))
    )
    print("dsls starting shape: ", gt1.data_subjects.shape, gt2.data_subjects.shape)
    return dot(gt1.data_subjects, gt2.data_subjects)

res3d = test_dot()
res3d.shape  # ideally: (4, 3, 4, 3, 7)

dsls starting shape:  (2, 3, 4, 54) (2, 3, 54, 7)
Target shapes:  (3, 4, 1, 1) (1, 1, 3, 7)
summed shapes:  (108, 3, 4, 1, 1) (108, 1, 1, 3, 7)
(108, 3, 4, 3, 7)
Projection shape:  (108, 3, 4, 3, 7) (108, 3, 4, 3, 7)
summed shapes after projection:  (108, 3, 4, 3, 7) (108, 3, 4, 3, 7)
output shape:  (216, 3, 4, 3, 7)
output shape after np.unique:  (4, 3, 4, 3, 7)


(4, 3, 4, 3, 7)

In [7]:
from syft import GammaTensor, lazyrepeatarray

def test_dot():
    
    # 2, 4, 54
    gt1 = GammaTensor(
        child=np.random.random((4, 54)), 
        data_subjects=DataSubjectList.from_objs(
            np.concatenate((
                np.random.choice(["Ishan"], (1, 4, 54)),
                np.random.choice(["John Cena"], (1, 4, 54))
            )),
        ),
        min_vals=lazyrepeatarray(0, (4, 54)) , 
        max_vals=lazyrepeatarray(1, (4, 54))
    )
    
    # 2, 54, 7
    gt2 = GammaTensor(
        child=np.random.random(( 54, 7)), 
        data_subjects=DataSubjectList.from_objs(
            np.concatenate(( 
                np.random.choice(["Hulk Hogan"], (1,  54, 7)),
                np.random.choice(["Potato"], (1, 54, 7))
            )),
        ),
        min_vals=lazyrepeatarray(0, (54, 7)) , 
        max_vals=lazyrepeatarray(1, (54, 7))
    )
    print("dsls starting shape: ", gt1.data_subjects.shape, gt2.data_subjects.shape)
    return dot(gt1.data_subjects, gt2.data_subjects)

res3d = test_dot()
res3d.shape  # ideally: (4, 4, 7)

dsls starting shape:  (2, 4, 54) (2, 54, 7)
Target shapes:  (4, 1, 1) (1, 1, 7)
summed shapes:  (108, 4, 1, 1) (108, 1, 1, 7)
(108, 4, 1, 7)
Projection shape:  (108, 4, 1, 7) (108, 4, 1, 7)
summed shapes after projection:  (108, 4, 1, 7) (108, 4, 1, 7)
output shape:  (216, 4, 1, 7)
output shape after np.unique:  (4, 4, 1, 7)


(4, 4, 1, 7)

<hr>
<hr>


## Dot Examples

In [8]:
# 1D
a = np.ones(5)
b = np.ones(5)

a.dot(b).shape

()

In [9]:
# 1D
a = np.ones(5) * 2
b = np.ones(5) * 3

a.dot(b)

30.0

In [10]:
# 2D
a = np.ones((5, 6))
b = np.ones((6, 7))

a.dot(b).shape

(5, 7)

In [11]:
# 2D x 1D
a = np.ones((5, 6))
b = np.ones((6))

a.dot(b).shape

(5,)

In [12]:
# 3D x 2D
a = np.ones((2,3,4))
b = np.ones((4, 5))

a.dot(b).shape

(2, 3, 5)

In [13]:
# 3D
a = np.ones((2,3,4))
b = np.ones((6,4,7))

a.dot(b).shape

(2, 3, 6, 7)

In [14]:
# 3D
a = np.ones((3, 4, 54))
b = np.ones((3, 54, 7))

a.dot(b).shape

(3, 4, 3, 7)

In [15]:
# 3D
a = np.ones((2,3,4,5,6))
b = np.ones((7,8,9,6,10))

a.dot(b).shape

(2, 3, 4, 5, 7, 8, 9, 10)

In [16]:
# 3D
a = np.ones((4,54))
b = np.ones((54,7))

a.dot(b).shape

(4, 7)

In [17]:
np.ones((108,3,4,7)) * np.ones((108,3,4,1))

array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]],


       [[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]],


       [[[1., 1., 1., ..

In [18]:
np.random.random((4, 1)) * np.ones((4, 7))

array([[0.05653496, 0.05653496, 0.05653496, 0.05653496, 0.05653496,
        0.05653496, 0.05653496],
       [0.08073315, 0.08073315, 0.08073315, 0.08073315, 0.08073315,
        0.08073315, 0.08073315],
       [0.70423316, 0.70423316, 0.70423316, 0.70423316, 0.70423316,
        0.70423316, 0.70423316],
       [0.6794775 , 0.6794775 , 0.6794775 , 0.6794775 , 0.6794775 ,
        0.6794775 , 0.6794775 ]])

In [19]:
gt1 = GammaTensor(
    child=np.random.random((4, 54)), 
    data_subjects=DataSubjectList.from_objs(
        np.concatenate((
            np.random.choice(["Ishan"], (1, 4, 54)),
            np.random.choice(["John Cena"], (1, 4, 54))
        )),
    ),
    min_vals=lazyrepeatarray(0, (4, 54)) , 
    max_vals=lazyrepeatarray(1, (4, 54))
)
    
# 2, 54, 7
gt2 = GammaTensor(
    child=np.random.random(( 54, 7)), 
    data_subjects=DataSubjectList.from_objs(
        np.concatenate(( 
            np.random.choice(["Hulk Hogan"], (1,  54, 7)),
            np.random.choice(["Potato"], (1, 54, 7))
        )),
    ),
    min_vals=lazyrepeatarray(0, (54, 7)) , 
    max_vals=lazyrepeatarray(1, (54, 7))
)
print("dsls starting shape: ", gt1.data_subjects.shape, gt2.data_subjects.shape)

dsls starting shape:  (2, 4, 54) (2, 54, 7)


In [20]:
res = gt1.dot(gt2)
res.shape, res.data_subjects.shape

target shapes:  (4, 1, 1) (1, 1, 7)
summed shapes:  (108, 4, 1, 1) (108, 1, 1, 7)
Projection shapes: (108, 4, 1, 7) (108, 4, 1, 7)
summed shapes:  (108, 4, 1, 7) (108, 4, 1, 7)
output shape: (216, 4, 1, 7)
output shape after unique: (4, 4, 7)


((4, 7), (4, 4, 7))

In [24]:
res_matmul = gt1 @ gt2

In [25]:
res_matmul.shape, res_matmul.data_subjects.shape

((4, 7), (4, 4, 7))

In [22]:
gt3 = GammaTensor(
    child=np.random.random((3, 4, 54)), 
    data_subjects=DataSubjectList.from_objs(
        np.concatenate((
            np.random.choice(["Ishan"], (1, 3, 4, 54)),
            np.random.choice(["John Cena"], (1, 3, 4, 54))
        )),
    ),
    min_vals=lazyrepeatarray(0, (3, 4, 54)) , 
    max_vals=lazyrepeatarray(1, (3, 4, 54))
)

# (2, 3, 54, 7)
gt4 = GammaTensor(
    child=np.random.random((3, 54, 7)), 
    data_subjects=DataSubjectList.from_objs(
        np.concatenate(( 
            np.random.choice(["Hulk Hogan"], (1, 3, 54, 7)),
            np.random.choice(["Potato"], (1, 3, 54, 7))
        )),
    ),
    min_vals=lazyrepeatarray(0, (3, 54, 7)) , 
    max_vals=lazyrepeatarray(1, (3, 54, 7))
)

res3d = gt3.dot(gt4)

target shapes:  (3, 4, 1, 1) (1, 1, 3, 7)
summed shapes:  (108, 3, 4, 1, 1) (108, 1, 1, 3, 7)
Projection shapes: (108, 3, 4, 3, 7) (108, 3, 4, 3, 7)
summed shapes:  (108, 3, 4, 3, 7) (108, 3, 4, 3, 7)
output shape: (216, 3, 4, 3, 7)
output shape after unique: (4, 3, 4, 3, 7)


In [23]:
res3d.shape, res3d.data_subjects.shape

((3, 4, 3, 7), (4, 3, 4, 3, 7))